In [3]:
import cv2
import numpy as np
import os
from sklearn import svm, metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from keras.utils import to_categorical



In [4]:
current_dir= os.getcwd()
folder_path = ""+ current_dir +"\\test"
print(folder_path)
ListimagesTest = []


folder_path_train= ""+ current_dir +"\\train"
ListimagesTrain = []


nbImagesNORMALTest = 0
nbImagesPNEUMONIATest = 0

nbImagesNORMALTrain = 0
nbImagesPNEUMONIATrain = 0

C:\Users\ferre\machine_learning\Dataset_IA\T_DEV_810_dataset\test


In [5]:
for filename in os.listdir(folder_path+'\\NORMAL'):
   
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        # Ouvrir l'image et l'ajouter à la liste
        img = cv2.imread(os.path.join(folder_path+'\\NORMAL', filename))
        nbImagesNORMALTest +=1
        ListimagesTest.append(img)
for filename in os.listdir(folder_path+'\\PNEUMONIA'): 
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        img = cv2.imread(os.path.join(folder_path+'\\PNEUMONIA', filename))
        nbImagesPNEUMONIATest +=1
        ListimagesTest.append(img)

In [ ]:
for filename in os.listdir(folder_path_train+'\\NORMAL'): 
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        img = cv2.imread(os.path.join(folder_path_train+'\\NORMAL', filename))
        nbImagesNORMALTrain+=1
        ListimagesTrain.append(img)
        
for filename in os.listdir(folder_path_train+'\\PNEUMONIA'):   
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"): 
        img = cv2.imread(os.path.join(folder_path_train+'\\PNEUMONIA', filename))
        nbImagesPNEUMONIATrain +=1
        ListimagesTrain.append(img)

In [ ]:
img0shape = (224, 224)
ListimagesTestResize = []
ListimagesTrainResize = []

for img in ListimagesTest :
    ListimagesTestResize.append(cv2.resize(img,img0shape))
    
    
for img in ListimagesTrain :
    ListimagesTrainResize.append(cv2.resize(img,img0shape))

In [ ]:
ListimagesTestNorm = []
ListimagesTrainNorm = []


for img in ListimagesTestResize:
    ListimagesTestNorm.append(cv2.normalize(img, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F))
for img in ListimagesTrainResize:
    ListimagesTrainNorm.append(cv2.normalize(img, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F))

In [ ]:
ListimagesTestNorm = np.array(ListimagesTestNorm).reshape(len(ListimagesTestNorm),224,224,3)
ListimagesTrainNorm= np.array(ListimagesTrainNorm).reshape(len(ListimagesTrainNorm),224,224,3)

In [ ]:
unTest = np.array(np.ones(nbImagesPNEUMONIATest))
zeroTest = np.array(np.zeros(nbImagesNORMALTest))

unTrain =np.array(np.ones(nbImagesPNEUMONIATrain))
zeroTrain = np.array(np.zeros(nbImagesNORMALTrain))

ListimagesTestNormResult = np.concatenate((zeroTest ,unTest) )
ListimagesTrainNormResult = np.concatenate((zeroTrain ,unTrain) )

In [ ]:
print(len(ListimagesTrainNormResult))
print(len(ListimagesTrainNorm))

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers


In [ ]:
# Création du modèle
model = tf.keras.Sequential()

# Ajout des couches
model.add(layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(layers.Conv2D(filters=256, kernel_size=(5, 5), activation='relu', padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), activation='relu', padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), activation='relu', padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(units=4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(units=4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(units=1, activation='sigmoid'))

# Compilation du modèle
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Affichage de la structure du modèle
model.summary()


In [ ]:
# Entraînement du modèle
# Mélanger les données d'entraînement

ListimagesTrainNorm = ListimagesTrainNorm
ListimagesTrainNormResult = ListimagesTrainNormResult

# Entraînement du modèle
model.fit(ListimagesTrainNorm, ListimagesTrainNormResult, epochs=10, batch_size=32, validation_data=(ListimagesTestNorm, ListimagesTestNormResult))


In [ ]:
score = model.evaluate(ListimagesTestNorm, ListimagesTestNormResult, verbose=0)
print('Perte (loss) :', score[0])
print('Précision (accuracy) :', score[1])

In [ ]:

y_pred = model.predict(ListimagesTestNorm)


In [ ]:
print(ListimagesTestNormResult)
print( y_pred)

In [ ]:
disp = metrics.ConfusionMatrixDisplay.from_predictions(ListimagesTestNormResult, y_pred)



In [ ]:
print(classification_report())